In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [3]:
vectorised_data_df = pd.read_feather('vectorised_data_df.feather') 
reviews_df = pd.read_hdf('causal_review_df_new_independent2.5.h5').iloc[:, : 20]

In [4]:
len(vectorised_data_df)

863

In [5]:
title_reviews_dict = {}
grouped_reviews = reviews_df.groupby('movie_title')
for title in vectorised_data_df['title'].tolist():
    reviews_by_title = reviews_df[reviews_df['movie_title'] == title]
    #print(reviews_by_title.columns)
    if len(reviews_by_title) != 0:
        title_reviews_dict[title] = reviews_by_title[['individual_meta_score','text','universal communicability',
       'a sheer personal preference based on desire or inclination','positivity in sentiment',
    'a sheer personal dispreference based on an absence of desire and inclination','allows for no one to disagree']]
    else:
        if ', The' in title:
            adjusted_title = title[-3:] + ' ' + title[:-5]
            reviews_by_title = reviews_df[reviews_df['movie_title'] == adjusted_title]
            if len(reviews_by_title) != 0:
                title_reviews_dict[title] = reviews_by_title[['individual_meta_score','text','universal communicability','positivity in sentiment',
       'a sheer personal preference based on desire or inclination','a sheer personal dispreference based on an absence of desire and inclination',
                                                              'allows for no one to disagree']]


In [6]:
script_review_df = vectorised_data_df[vectorised_data_df['title'].isin(list(title_reviews_dict.keys()))]
script_review_df['reviews'] = script_review_df['title'].map(title_reviews_dict) 

C:\Users\balvi\AppData\Local\Temp\ipykernel_3892\2169358641.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  script_review_df['reviews'] = script_review_df['title'].map(title_reviews_dict)


In [7]:
len(script_review_df)

776

In [8]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import gc

gc.collect()
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased',
                                        torch_dtype=torch.bfloat16,attn_implementation="flash_attention_2").to(device)

model.half()  # Convert model to half precision

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertFlashAttention2(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2):

In [9]:
def encode(text, model, tokenizer, device):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze()  # Taking the first token ([CLS]) embedding

In [10]:
def encode_reviews(row, model, tokenizer, device):
    vector_list = []
    for review in row['reviews']['text']:
        vector = encode(review, model, tokenizer, device)
        vector_list.append(vector.cpu().numpy())  # Move vectors to CPU to save GPU memory
        torch.cuda.empty_cache()  # Frees up unutilized GPU memory
    row['reviews']['review_vector'] = vector_list
    return row

In [11]:
def vector_projection(a, b):
    """Project vector a onto vector b using numpy."""
    dot_product = np.dot(a, b)
    norm_b_squared = np.dot(b, b)
    return (dot_product / norm_b_squared) * b

In [12]:
tqdm.pandas()

In [13]:
vectorised_script_review_df = script_review_df.progress_apply(lambda x: encode_reviews(x, model, tokenizer, device), axis=1)

  0%|          | 0/776 [00:00<?, ?it/s]

In [14]:
script_lines_projected_vectors_list = []
for _, data in tqdm(vectorised_script_review_df.iterrows()):
    line_projected_vectors_list = []
    for line, line_vector in list(zip(data['split_script'], data['script_vectors'])):
        projected_vectors_list = []
        for _,review_data in data['reviews'].iterrows():
            normalised_line_vector = line_vector/np.linalg.norm(line_vector)
            normalised_review_vector = review_data['review_vector']/np.linalg.norm(review_data['review_vector'])
            projected_vector = vector_projection(normalised_line_vector, normalised_review_vector)
            projected_vectors_list.append(np.linalg.norm(projected_vector))
        line_projected_vectors_list.append(projected_vectors_list)
    script_lines_projected_vectors_list.append(line_projected_vectors_list)
            

0it [00:00, ?it/s]

In [15]:
vectorised_script_review_df['projected_magnitudes'] = script_lines_projected_vectors_list

In [16]:
vectorised_script_review_df['title']

0                             10 Things I Hate About You
1                                                     12
4                                       12 Years a Slave
5                                              127 Hours
6                             1492: Conquest of Paradise
8                                               17 Again
10                                 2001: A Space Odyssey
11                                                  2012
13                                    30 Minutes or Less
14                                                    42
15                                         44 Inch Chest
16                                               48 Hrs.
19                                                   8MM
20                                                     9
21                                        A Few Good Men
22                                   A Most Violent Year
23                                  A Prayer Before Dawn
24                             

In [20]:
row = vectorised_script_review_df[vectorised_script_review_df['title'] == "Boyhood"].squeeze(0)

In [21]:
row['reviews']

,individual_meta_score,text,universal communicability,a sheer personal preference based on desire or inclination,positivity in sentiment,a sheer personal dispreference based on an absence of desire and inclination,allows for no one to disagree,review_vector
32185,100,"There’s not a great theme, a great performance or even a great scene in Boyhood. But I think it might be a great picture.",0.053572,0.676268,0.141920,0.644752,0.005038,"[0.00279, -0.1608, 0.0283, -0.0665, -0.04617, -0.3237, 0.1119, 0.4316, -0.1204, -0.159, -0.01386, -0.06616, 0.1385, 0.3542, -0.011635, 0.1641, 0.000988, 0.06586, 0.2037, -0.1976, -0.1897, -0.06604, -0.0763, 0.0891, -0.1207, -0.2399, 0.1758, -0.06216, 0.2964, -0.01297, -0.04712, 0.08246, -0.358, -0.2517, -0.002987, -0.2147, 0.02623, -0.1981, 0.02966, -0.1804, 0.08606, 0.03528, 0.0215, -0.0769, -0.129, -0.03006, -2.355, 0.000529, 0.004738, -0.303, 0.2443, 0.05728, 0.3564, 0.3188, 0.2316, 0.2216, -0.2361, 0.4624, -0.0319, 0.11993, 0.1157, -0.0059, -0.02411, -0.11224, -0.0564, 0.0436, 0.02313, 0.1407, -0.1506, 0.2322, -0.1703, -0.1544, 0.2208, -0.3127, 0.05902, 0.04092, 0.1068, 0.1953, -0.1704, 0.0777, 0.06235, 0.3352, 0.16, 0.1697, -0.05487, 0.0923, -0.06046, -0.12427, 0.2834, 0.3892, -0.413, -0.1597, 0.0481, 0.2439, 0.514, -0.3413, 0.02121, -0.08154, 0.1975, 0.322, ...]"
32186,100,"The greatest movies, the ones that stick with us, are those that hold up a mirror to the human condition and reflect something back at us that we too often manage to overlook. Boyhood is one of those movies, and with it Linklater proves he is among the best practitioners of that art.",0.839547,0.083272,0.003778,0.005000,0.345220,"[0.1609, 0.05338, -0.3076, -0.09766, -0.00289, -0.2467, 0.136, 0.2734, -0.10974, -0.2651, 0.04956, -0.075, 0.04434, 0.3257, 0.1598, 0.1052, 0.02744, 0.03342, 0.1117, -0.108, -0.355, -0.1312, 0.2212, 0.2052, -0.1902, -0.05362, -0.12256, -0.0883, 0.0898, 0.0953, 0.1797, 0.1833, -0.2698, -0.4216, -0.01662, -0.323, 0.1556, -0.114, 0.1744, 0.1788, -0.01263, 0.2727, 0.11304, -0.2415, -0.1476, -0.1993, -2.846, 0.04416, -0.2717, -0.12164, 0.1847, 0.2014, 0.108, 0.3142, 0.3977, 0.5664, -0.405, 0.391, -0.06335, 0.09296, 0.3188, 0.08673, -0.12067, -0.06696, -0.1786, 0.204, -0.1307, 0.1425, -0.1426, 0.466, -0.2169, -0.1732, 0.4236, -0.11816, 0.1648, 0.01481, 0.0405, 0.0434, -0.2008, -0.0578, 0.127, 0.3345, 0.1849, 0.3408, 0.01196, 0.235, 0.1142, 0.009, 0.1716, 0.3545, -0.348, -0.1637, 0.1139, 0.302, 0.605, -0.5513, 0.1534, -0.10095, -0.0693, 0.3691, ...]"
32187,100,The film would be incalculably different if the lead role had been divided between two or three young actors for a conventional shoot. But Linklater’s patience allows us to see a thoughtful personality being formed both on and off the screen.,0.394183,0.283687,0.357710,0.014962,0.171230,"[0.06824, -0.10034, -0.1742, 0.0049, 0.18, -0.003136, 0.2164, 0.03992, 0.04623, -0.1621, 0.1947, 0.13, 0.1467, 0.3042, 0.02832, 0.18, 0.0737, 0.09937, 0.1967, -0.286, -0.113, -0.2302, 0.11505, 0.2522, -0.1553, -0.1471, 0.0816, 0.01694, 0.1339, -0.00828, 0.1373, -0.02121, -0.365, -0.256, 0.01868, -0.224, 0.0284, 0.0674, 0.04007, -0.0984, 0.04074, 0.2534, 0.0591, -0.2156, -0.1329, -0.1533, -2.607, -0.0012045, -0.1545, -0.3066, 0.2207, 0.2217, 0.4197, 0.2388, 0.2754, 0.251, -0.2888, 0.3752, 0.007065, 0.194, 0.0414, 0.0199, 0.05862, -0.0655, -0.01811, 0.1162, -0.11475, -0.01152, -0.0534, 0.3962, -0.1936, -0.1699, 0.08527, -0.1565, -0.0625, -0.1884, 0.0382, 0.2974, -0.1926, -0.02069, 0.2063, 0.2064, -0.01486, 0.1293, -0.0948, 0.1167, 0.199, -0.0418, 0.378, 0.486, -0.4314, -0.1725, -0.07465, 0.0989, 0.5005, -0.3318, 0.2322, 0.0452, 0.1521, 0.3845, ...]"
32188,100,"A home movie of a fictional home life, an epic assembled from vignettes, Boyhood shimmers with unforced reality. It shows how an ordinary life can be reflected in an extraordinary movie.",0.981731,0.017478,0.524910,0.001098,0.203928,"[-0.06384, -0.2491, 0.0379, 0.00503, 0.1364, -0.2676, 0.1777, 0.31, 0.1231, -

In [22]:
line_review_magnitudes_dict = {}
for line,review_magnitudes in list(zip(row['split_script'], row['projected_magnitudes'])):
    line_review_magnitudes_dict[line] = review_magnitudes

In [24]:
for i, review in enumerate(row['reviews'].iterrows()):
    print(review[1]['individual_meta_score'], review[1]['text'])
    most_influenced_line_magnitude_list = []
    for line, magnitudes in line_review_magnitudes_dict.items():
        most_influenced_line_magnitude_list.append((line, magnitudes[i]))
    most_influenced_line_magnitude_list_sorted = sorted(most_influenced_line_magnitude_list,key=lambda x: x[1], reverse=True)
    print(most_influenced_line_magnitude_list_sorted[0:3])

100 There’s not a great theme, a great performance or even a great scene in Boyhood. But I think it might be a great picture.
[("\n                      DAD\r\n          I know, I know, right? It's gonna be\r\n          uh, fifteen years till I have an\r\n          empty nest. But hey, I'd love to\r\n          pitch in, help with this, if I could.\r", 0.955), ("\n            MASON\r\nShe wasn't a silly girl, though.   I\r\nmean, she's a serious person. I\r\nreally thought we were --\r", 0.954), ("\n                       TEACHER\r\n           Well, it's time to finish it.\r", 0.9536)]
100 The greatest movies, the ones that stick with us, are those that hold up a mirror to the human condition and reflect something back at us that we too often manage to overlook. Boyhood is one of those movies, and with it Linklater proves he is among the best practitioners of that art.
[('\n                      MOM (O.S.)\r\n          His cognitive series, his interviews\r\n          with orphans, Etho

In [65]:
from sklearn.metrics.pairwise import cosine_similarity

In [210]:
art_vector = encode('art', model, tokenizer, device).cpu()
entertainment_vector = encode('entertainment', model, tokenizer, device).cpu()
#odyssey_vector = vectorised_script_review_df[vectorised_script_review_df['title'] == '2001: A Space Odyssey'].squeeze(0)['script_vectors'].sum()
#ventura_vector = vectorised_script_review_df[vectorised_script_review_df['title'] == 'Ace Ventura: Pet Detective'].squeeze(0)['script_vectors'].sum()
result_list = []
for i, data in tqdm(vectorised_script_review_df.iterrows()):
    projected_magnitudes_scripts = np.array(data['projected_magnitudes']).T
    for projection_magnitudes, (review_columns,review) in list(zip(projected_magnitudes_scripts, data['reviews'].iterrows())):
        script_weighted_sum = np.dot(projection_magnitudes, data['script_vectors'])
        first_comparision = cosine_similarity(art_vector.reshape(1, -1), script_weighted_sum.reshape(1, -1)).item()
        second_comparision = cosine_similarity(entertainment_vector.reshape(1, -1), script_weighted_sum.reshape(1, -1)).item()
        result_list.append({**review.to_dict(), **{'title':data['title'], 'first_comparison':first_comparision, 'second_comparison':second_comparision}})
    

0it [00:00, ?it/s]

In [211]:
result_df = pd.DataFrame(result_list)

In [213]:
grouped_result_df = result_df.groupby('title')[['universal communicability', 'first_comparison', 'second_comparison']].describe()

In [220]:
corr_df = pd.DataFrame(list(zip(grouped_result_df.index, grouped_result_df['universal communicability']['mean'],grouped_result_df['first_comparison']['mean']))).set_index(0)

In [223]:
import pingouin as pg

C:\Users\balvi\anaconda3\Lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [224]:
pg.corr(corr_df[1], corr_df[2])

,n,r,CI95%,p-val,BF10,power
pearson,776,0.013844,"[-0.06, 0.08]",0.700208,0.048,0.067137
